In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

rng = np.random.default_rng(42)
n = 2000


tenure = rng.integers(1, 61, size=n)
monthly_charges = rng.normal(50, 15, size=n)
num_services = rng.integers(1, 6, size=n)
support_calls = rng.integers(0, 11, size=n)

base_score = (
    0.04 * monthly_charges
    - 0.03 * tenure
    + 0.2  * support_calls
    - 0.1  * num_services
)


noise = rng.normal(0, 3, size=n)
logits = base_score + noise
prob_churn = 1 / (1 + np.exp(-logits))

churn = (prob_churn > 0.5).astype(int)

df = pd.DataFrame({
    'tenure': tenure,
    'monthly_charges': monthly_charges,
    'num_services': num_services,
    'support_calls': support_calls,
    'churn': churn
})
df.head()

,tenure,monthly_charges,num_services,support_calls,churn
0,6,38.867119,1,0,1
1,47,63.865366,4,4,1
2,40,50.519178,4,7,0
3,27,45.758064,4,3,1
4,26,48.407271,5,9,1


In [ ]:
X = df[['tenure', 'monthly_charges', 'num_services', 'support_calls']].values
y = df['churn'].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
from tensorflow.keras.layers import Input

model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(16, activation='relu'),
    Dense(8, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=20,
    batch_size=32,
    verbose=1
)

loss, acc = model.evaluate(X_test, y_test, verbose=0)
print(f'Точность на тестовых данных: {acc:.2f}')

import pickle
model.save('churn_model.h5')
with open('churn_scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)
print('Модель и scaler сохранены!')

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.7307 - loss: 0.6409
Epoch 2/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7315 - loss: 0.6077
Epoch 3/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7196 - loss: 0.5876
Epoch 4/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7256 - loss: 0.5770
Epoch 5/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7105 - loss: 0.5759
Epoch 6/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7234 - loss: 0.5698
Epoch 7/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7403 - loss: 0.5508
Epoch 8/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7346 - loss: 0.5500
Epoch 9/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7472 - loss: 0.5312
Epoch 10/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7255 - loss: 0.5627
Epoch 11/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7105 - loss: 0.5737
Epoch 12/20
44/44 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7238 - lo

In [ ]:
example_customers = pd.DataFrame({
    'tenure': [5, 45, 12, 30, 2, 55],
    'monthly_charges': [85.0, 35.0, 70.0, 50.0, 90.0, 40.0],
    'num_services': [2, 4, 1, 3, 5, 4],
    'support_calls': [8, 1, 5, 2, 10, 0],
    'description': [
        'Новый клиент, высокая цена, много жалоб',
        'Долгосрочный клиент, низкая цена, мало жалоб',
        'Средний срок, высокая цена, средние жалобы',
        'Средний срок, средняя цена, мало жалоб',
        'Очень новый, очень высокая цена, много жалоб',
        'Очень долгосрочный, низкая цена, нет жалоб'
    ]
})

example_customers


In [ ]:
X_examples = example_customers[['tenure', 'monthly_charges', 'num_services', 'support_calls']].values
X_examples_scaled = scaler.transform(X_examples)

predictions = model.predict(X_examples_scaled, verbose=0)
probabilities = predictions.flatten()

example_customers['churn_probability'] = probabilities
example_customers['predicted_churn'] = (probabilities > 0.5).astype(int)
example_customers['status'] = example_customers['predicted_churn'].map({0: 'Остается', 1: 'Уходит'})

print('='*70)
print('ПРЕДСКАЗАНИЯ ОТТОКА КЛИЕНТОВ')
print('='*70)
print()

for idx, row in example_customers.iterrows():
    print(f"Клиент {idx + 1}: {row['description']}")
    print(f"  Стаж: {row['tenure']} месяцев")
    print(f"  Ежемесячная плата: ${row['monthly_charges']:.2f}")
    print(f"  Количество услуг: {int(row['num_services'])}")
    print(f"  Обращений в поддержку: {int(row['support_calls'])}")
    print(f"  Вероятность оттока: {row['churn_probability']:.1%}")
    print(f"  Предсказание: {row['status']}")
    print()


In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].bar(range(len(example_customers)), example_customers['churn_probability'] * 100, 
            color=['red' if p > 0.5 else 'green' for p in example_customers['churn_probability']])
axes[0].axhline(y=50, color='black', linestyle='--', linewidth=1, label='Порог 50%')
axes[0].set_xlabel('Клиент')
axes[0].set_ylabel('Вероятность оттока (%)')
axes[0].set_title('Вероятность оттока по клиентам')
axes[0].set_xticks(range(len(example_customers)))
axes[0].set_xticklabels([f'Клиент {i+1}' for i in range(len(example_customers))], rotation=45)
axes[0].legend()
axes[0].grid(axis='y', alpha=0.3)

churn_count = example_customers['predicted_churn'].sum()
stay_count = len(example_customers) - churn_count
axes[1].pie([stay_count, churn_count], labels=['Остаются', 'Уходят'], 
            autopct='%1.1f%%', colors=['green', 'red'], startangle=90)
axes[1].set_title('Распределение предсказаний')

plt.tight_layout()
plt.show()

print(f'\nИтого из {len(example_customers)} клиентов:')
print(f'  - Остаются: {stay_count} ({stay_count/len(example_customers):.1%})')
print(f'  - Уходят: {churn_count} ({churn_count/len(example_customers):.1%})')


In [ ]:
high_risk = example_customers[example_customers['predicted_churn'] == 1]
low_risk = example_customers[example_customers['predicted_churn'] == 0]

print('='*70)
print('АНАЛИЗ ФАКТОРОВ ОТТОКА')
print('='*70)
print()

if len(high_risk) > 0:
    print('Клиенты с высоким риском оттока (уходят):')
    print(f"  Средний стаж: {high_risk['tenure'].mean():.1f} месяцев")
    print(f"  Средняя ежемесячная плата: ${high_risk['monthly_charges'].mean():.2f}")
    print(f"  Среднее количество услуг: {high_risk['num_services'].mean():.1f}")
    print(f"  Среднее количество обращений: {high_risk['support_calls'].mean():.1f}")
    print()

if len(low_risk) > 0:
    print('Клиенты с низким риском оттока (остаются):')
    print(f"  Средний стаж: {low_risk['tenure'].mean():.1f} месяцев")
    print(f"  Средняя ежемесячная плата: ${low_risk['monthly_charges'].mean():.2f}")
    print(f"  Среднее количество услуг: {low_risk['num_services'].mean():.1f}")
    print(f"  Среднее количество обращений: {low_risk['support_calls'].mean():.1f}")
    print()

print('Выводы:')
print('  - Клиенты с коротким стажем и высокими ценами чаще уходят')
print('  - Много обращений в поддержку увеличивает риск оттока')
print('  - Долгосрочные клиенты с низкими ценами реже уходят')
